In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/10623 Project")

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2025-12-06 21:12:00--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.2.206, 52.217.207.25, 3.5.2.152, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.2.206|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip.2’

val2017.zip.2         8%[>                   ]  67.10M  35.4MB/s               ^C
--2025-12-06 21:12:02--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.19, 16.15.196.203, 54.231.232.73, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip.2’

       annotations_   4%[                    ]  10.20M  14.9MB/s               ^C


In [ ]:
!unzip val2017.zip

Streaming output truncated to the last 5000 lines.
 extracting: val2017/000000212226.jpg  
 extracting: val2017/000000231527.jpg  
 extracting: val2017/000000578922.jpg  
 extracting: val2017/000000062808.jpg  
 extracting: val2017/000000119038.jpg  
 extracting: val2017/000000114871.jpg  
 extracting: val2017/000000463918.jpg  
 extracting: val2017/000000365745.jpg  
 extracting: val2017/000000320425.jpg  
 extracting: val2017/000000481404.jpg  
 extracting: val2017/000000314294.jpg  
 extracting: val2017/000000335328.jpg  
 extracting: val2017/000000513688.jpg  
 extracting: val2017/000000158548.jpg  
 extracting: val2017/000000132116.jpg  
 extracting: val2017/000000415238.jpg  
 extracting: val2017/000000321333.jpg  
 extracting: val2017/000000081738.jpg  
 extracting: val2017/000000577584.jpg  
 extracting: val2017/000000346905.jpg  
 extracting: val2017/000000433980.jpg  
 extracting: val2017/000000228144.jpg  
 extracting: val2017/000000041872.jpg  
 extracting: val2017/00000011

In [ ]:
!unzip annotations_trainval2017.zip

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


In [ ]:
!pip install controlnet_aux
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39

In [ ]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

from diffusers import StableDiffusionXLAdapterPipeline, T2IAdapter, EulerAncestralDiscreteScheduler, AutoencoderKL
from diffusers.utils import load_image, make_image_grid
from controlnet_aux import ZoeDetector
import torch

/usr/local/lib/python3.12/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segmen

In [ ]:
IMAGE_EXTENSIONS = {".png", ".jpg", ".jpeg", ".bmp", ".webp"}

def iter_images(folder):
    for p in folder.iterdir():
        if p.is_file() and p.suffix.lower() in IMAGE_EXTENSIONS:
            yield p

In [ ]:
input_dir = Path("/content/drive/MyDrive/Colab Notebooks/10623 Project/val2017")
output_dir = Path("/content/drive/MyDrive/Colab Notebooks/10623 Project/depth2017")

adapter = T2IAdapter.from_pretrained(
  "TencentARC/t2i-adapter-depth-zoe-sdxl-1.0", torch_dtype = torch.float16, varient = "fp16"
).to("cuda")

model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
euler_a = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder = "scheduler")
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype = torch.float16)

pipe = StableDiffusionXLAdapterPipeline.from_pretrained(
    model_id, vae = vae, adapter = adapter, scheduler = euler_a, torch_dtype = torch.float16, variant = "fp16"
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()

zoe_depth = ZoeDetector.from_pretrained(
    "valhalla/t2iadapter-aux-models", filename = "zoed_nk.pth", model_type = "zoedepth_nk"
).to("cuda")


files = list(iter_images(input_dir))

for img_path in tqdm(files):
    image = Image.open(img_path).convert("RGB")
    depth_img = zoe_depth(image, gamma_corrected = True, detect_resolution = 512, image_resolution = 1024)
    depth_img = depth_img.convert("L")
    out_path = output_dir / f"{img_path.stem}_depth.png"
    depth_img.save(out_path)

print(f"Saved {len(files)} depth maps to {output_dir}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/316M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

...

Using pretrained resource url::https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_NK.pt
Downloading: "https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_NK.pt" to /root/.cache/torch/hub/checkpoints/ZoeD_M12_NK.pt


100%|██████████| 1.35G/1.35G [00:14<00:00, 99.8MB/s]


Loaded successfully


  0%|          | 0/5000 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/controlnet_aux/zoe/__init__.py:79: RuntimeWarning: invalid value encountered in power
  depth = np.power(depth, 2.2)
/usr/local/lib/python3.12/dist-packages/controlnet_aux/zoe/__init__.py:80: RuntimeWarning: invalid value encountered in cast
  depth_image = (depth * 255.0).clip(0, 255).astype(np.uint8)
100%|██████████| 5000/5000 [33:57<00:00,  2.45it/s]

Saved 5000 depth maps to /content/drive/MyDrive/Colab Notebooks/10623 Project/depth2017


In [ ]:
image_dir = Path("/content/drive/MyDrive/Colab Notebooks/10623 Project/val2017")
sketch_dir = Path("/content/drive/MyDrive/Colab Notebooks/10623 Project/coco17sketches")
depth_dir = Path("/content/drive/MyDrive/Colab Notebooks/10623 Project/depth2017")

image_files = sorted(list(iter_images(image_dir)))
sketch_files = sorted(list(iter_images(sketch_dir)))
depth_files = sorted(list(iter_images(depth_dir)))

In [ ]:
import json

def get_file_id(filename):
    return os.path.splitext(filename)[0].split("_")[0].lstrip("0")

all_image_ids = set()
for image_path in tqdm(image_files):
    image_id = get_file_id(os.path.basename(image_path))
    all_image_ids.add(int(image_id))

# print(all_image_ids)
# print(len(all_image_ids))

annotation_file = "/content/drive/MyDrive/Colab Notebooks/10623 Project/annotations/captions_val2017.json"

with open(annotation_file, "r", encoding = "utf-8") as f:
    data = json.load(f)

annotations = data.get("annotations", data if isinstance(data, list) else [])

captions = {}
for ann in annotations:
    ann_image_id = ann.get("image_id")
    if ann_image_id in all_image_ids and ann_image_id not in captions:
        captions[ann_image_id] = ann.get("caption")

sorted_captions = sorted(captions.items())

100%|██████████| 5000/5000 [00:00<00:00, 198319.75it/s]


In [ ]:
hf_list = []

for caption, image_path, sketch_path, depth_path in tqdm(zip(sorted_captions, image_files, sketch_files, depth_files)):
    image_caption = caption[1]
    image_filename = os.path.basename(image_path)
    sketch_filename = os.path.basename(sketch_path)
    depth_filename = os.path.basename(depth_path)
    # print(str(caption[0]), get_file_id(image_filename), get_file_id(sketch_filename), get_file_id(depth_filename))
    # print(type(caption[0]), type(get_file_id(image_filename)), type(get_file_id(sketch_filename)), type(get_file_id(depth_filename)))
    # print(str(caption[0]) != get_file_id(image_filename))
    # print(get_file_id(image_filename) != get_file_id(sketch_filename))
    # print(get_file_id(sketch_filename) != get_file_index(depth_filename))
    if str(caption[0]) != get_file_id(image_filename) or get_file_id(image_filename) != get_file_id(sketch_filename) or get_file_id(sketch_filename) != get_file_id(depth_filename):
        continue
    hf_list.append({
        "image_id": caption[0],
        "text": image_caption,
        "image": image_path,
        "sketch": sketch_path,
        "depth": depth_path
    })
    # print(image_caption, image_filename, sketch_filename, depth_filename)
    # print(image_caption, image_path, sketch_path, depth_path)

5000it [00:00, 12502.20it/s]


In [ ]:
from datasets import Dataset, Features, Value, Image
from huggingface_hub import login

def to_path_str(v):
    return str(v) if isinstance(v, Path) else v

features = Features(
    {
        "image_id": Value("int64"),
        "text": Value("string"),
        "image": Image(),
        "sketch": Image(),
        "depth": Image()
    }
)

def gen(entries):
    for idx, e in enumerate(entries):
        yield {
            "image_id": e["image_id"],
            "text": e["text"],
            "image": to_path_str(e["image"]),
            "sketch": to_path_str(e["sketch"]),
            "depth": to_path_str(e["depth"]),
        }

ds = Dataset.from_generator(
    gen,
    features = features,
    gen_kwargs = {"entries": hf_list},
    split = "val"
)


print(ds)
print(ds[0]) if len(ds) else None


HF_TOKEN = "hf_**"
login(token = HF_TOKEN)
repo_id = "hengyiqun/10623-project-coco17"
ds.push_to_hub(repo_id, private = False, token = HF_TOKEN)

print(f"Done! Dataset uploaded to https://huggingface.co/datasets/{repo_id}")

Generating val split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['image_id', 'text', 'image', 'sketch', 'depth'],
    num_rows: 5000
})
{'image_id': 139, 'text': 'A woman stands in the dining area at the table.', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x426 at 0x7F73ED005850>, 'sketch': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1536x1024 at 0x7F73ECFE7AA0>, 'depth': <PIL.PngImagePlugin.PngImageFile image mode=L size=1536x1024 at 0x7F73EE491EE0>}


Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ? shards/s]

Map:   0%|          | 0/1667 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   3%|3         | 16.7MB /  486MB            

Done! Dataset uploaded to https://huggingface.co/datasets/hengyiqun/10623-project-coco17
